In [1]:
import langchain
print(langchain.__version__)

0.3.24


In [2]:
from dotenv import load_dotenv
# Adjust path if notebook isn't in 'notebook/' dir relative to root .env
loaded = load_dotenv("../.env")
print(f"Dotenv loaded: {loaded}")

Dotenv loaded: True


In [3]:
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage

# Initialize LLM (needs GROQ_API_KEY)
llm = ChatGroq(model="llama3-8b-8192") # Or your chosen model/provider

# Send a message directly
response = llm.invoke([HumanMessage(content="Tell me a joke")])

print(response.content)

Here's one:

Why couldn't the bicycle stand up by itself?

(wait for it...)

Because it was two-tired!

Hope that made you smile!


In [6]:
response

AIMessage(content="Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 14, 'total_tokens': 46, 'completion_time': 0.026666667, 'prompt_time': 0.002999473, 'queue_time': 0.235684042, 'total_time': 0.02966614}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_dadc9d6142', 'finish_reason': 'stop', 'logprobs': None}, id='run-a1aa3111-9147-421c-aea8-f646faddfc7c-0', usage_metadata={'input_tokens': 14, 'output_tokens': 32, 'total_tokens': 46})

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
print(output_parser.invoke(response)) # Parses the AIMessage object's content

Here's one:

Why couldn't the bicycle stand up by itself?

(wait for it...)

Because it was two-tired!

Hope that made you smile!


In [7]:
# Chain: Input -> LLM -> Parse Output to String
basic_chain = llm | output_parser

llm_response = basic_chain.invoke("Tell me a joke!") # Input is passed to llm
print(llm_response)

Here's one:

Why couldn't the bicycle stand up by itself?

(wait for it...)

Because it was two-tired!

Hope that made you smile!


In [8]:
from langchain_core.prompts import ChatPromptTemplate

# Template with a placeholder {topic}
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# See the prompt structure
print(prompt.invoke({"topic": "programming"}))

messages=[HumanMessage(content='Tell me a short joke about programming', additional_kwargs={}, response_metadata={})]


In [9]:
# Chain the prompt template with the basic LLM chain
joke_chain = prompt | llm | output_parser

# Invoke with a specific topic
programmer_joke = joke_chain.invoke({"topic": "programmer"})
print(programmer_joke)

Here's one:

Why did the programmer quit his job?

Because he didn't get arrays! (get a raise)


In [10]:
from langchain_core.messages import SystemMessage

system_message = SystemMessage(content="You are a helpful assistant that tells jokes.")
human_message = HumanMessage(content="Tell me about programming")

# Direct invocation with system/human message list
response_persona = llm.invoke([system_message, human_message])
print("Direct invoke with persona:\n", response_persona.content)

Direct invoke with persona:
 Programming! It's like trying to solve a puzzle blindfolded while being attacked by a swarm of bees. Just kidding, it's actually really fun once you get the hang of it!

But seriously, programming is like writing a recipe for your computer. You take some ingredients (like variables and functions), mix them together in a certain way (using syntax and logic), and voila! You get a delicious program that can do all sorts of cool things.

Speaking of cool things, did you hear about the programmer who quit his job because he didn't get arrays? (get it? arrays? ahaha)

Okay, okay, I'll stop with the jokes for now. But seriously, programming is a great skill to have, and there are so many resources available to help you learn. Whether you're interested in web development, game development, or just want to automate some tasks, there's a programming language out there for you.

So, what kind of programming are you interested in? I can give you some resources and tips

In [11]:
# More common: Use ChatPromptTemplate with roles
template_persona = ChatPromptTemplate(
    [
        ("system", "You are a helpful assistant that tells jokes."),
        ("human", "Tell me about {topic}")
    ]
)
# See the structured prompt
prompt_value_persona = template_persona.invoke({"topic": "programming"})
print(prompt_value_persona)

messages=[SystemMessage(content='You are a helpful assistant that tells jokes.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about programming', additional_kwargs={}, response_metadata={})]


In [12]:
# Chain with the persona template
persona_chain = template_persona | llm | output_parser
response_persona_chain = persona_chain.invoke({"topic": "programming"})
print("\nChain invoke with persona:")
print(response_persona_chain)


Chain invoke with persona:
Programming! It's a real "byte"-sized challenge, isn't it? But don't worry, I'm here to help you "debug" your understanding!

Programming is like cooking a recipe. You need to follow the steps, add the right ingredients, and stir it up just right to get the desired result. But instead of flour, sugar, and eggs, you're working with code, algorithms, and data structures!

Here's a joke to "compile" your thoughts:

Why do programmers prefer dark mode?

Because light attracts bugs!


In [13]:
from typing import List
from pydantic import BaseModel, Field

# Define the desired output structure
class MobileReview(BaseModel):
    phone_model: str = Field(description="Name and model of the phone")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.
Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.
Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

try:
    # Create an LLM variant that outputs in the MobileReview format
    structured_llm = llm.with_structured_output(MobileReview)
    output_obj = structured_llm.invoke(review_text)
    print("Structured Output Object:", output_obj)
    print("\nPros:", output_obj.pros)
except Exception as e:
    print(f"Structured output failed: {e}") # Might depend on LLM's ability

Structured Output Object: phone_model='Galaxy S21' rating=4.0 pros=['gorgeous screen', 'colors pop', 'insane camera'] cons=['pricey', 'no charger included', 'new button layout takes getting used to'] summary='Solid phone, but a few annoying quirks keep it from being perfect'

Pros: ['gorgeous screen', 'colors pop', 'insane camera']


In [14]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    print(f"Loading documents from: {folder_path}")
    if not os.path.isdir(folder_path): return []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        loader = None
        if filename.endswith('.pdf'): loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'): loader = Docx2txtLoader(file_path)
        else: print(f"  Skipping unsupported file type: {filename}"); continue
        try:
            loaded_docs = loader.load(); documents.extend(loaded_docs)
            print(f"    Loaded {filename} ({len(loaded_docs)} parts).")
        except Exception as e: print(f"    Error loading {filename}: {e}")
    return documents

# Adjust path as needed
folder_path = "data/"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents total.")

Loading documents from: data/
    Loaded company_profile.pdf (1 parts).
    Loaded employee_handbook.pdf (1 parts).
    Loaded it_support_policy.pdf (1 parts).
    Loaded meeting_guidelines.pdf (1 parts).
    Loaded product_faq.pdf (1 parts).
Loaded 5 documents total.


In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, length_function=len
)

if documents: # Only split if documents were loaded
    splits = text_splitter.split_documents(documents)
    print(f"Split the documents into {len(splits)} chunks.")
else:
    splits = []
    print("No documents to split.")

if documents: print("\nOriginal Document 0 Metadata:", documents[0].metadata)
if splits: print("\nFirst Split Chunk Metadata:", splits[0].metadata)
if splits: print("\nFirst Split Chunk Content:", splits[0].page_content[:300] + "...")

Split the documents into 5 chunks.

Original Document 0 Metadata: {'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250418155508', 'source': 'data/company_profile.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}

First Split Chunk Metadata: {'producer': 'PyFPDF 1.7.2 http://pyfpdf.googlecode.com/', 'creator': 'PyPDF', 'creationdate': 'D:20250418155508', 'source': 'data/company_profile.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}

First Split Chunk Content: Company Profile
Company Name: FutureTech Corp
Founded: 2012
Headquarters: San Francisco, CA
Employees: 1,200+
Mission:
To innovate intelligent solutions that simplify lives and empower industries through AI
and automation.
Flagship Products:
- Vision360 AI Camera
- AutoInsights Business Dashboard
- ...


In [16]:
# NOTE: This uses Nomic Embeddings, requires NOMIC_API_KEY
from langchain_nomic import NomicEmbeddings

embedding_function_nomic = None # Initialize
if splits:
    try:
        print("Initializing NOMIC Embeddings (requires NOMIC_API_KEY)...")
        embeddings_nomic = NomicEmbeddings(model="nomic-embed-text-v1.5")
        print("Nomic Embeddings Initialized.")
        # Example embedding one chunk
        # first_chunk_embedding = embeddings_nomic.embed_documents([splits[0].page_content])
        # print(f"Example vector length: {len(first_chunk_embedding[0])}")
        embedding_function_nomic = embeddings_nomic # Store for use
    except Exception as e:
        print(f"ERROR initializing Nomic Embeddings: {e}")
else:
    print("Skipping embeddings: No chunks.")

Initializing NOMIC Embeddings (requires NOMIC_API_KEY)...
Nomic Embeddings Initialized.


In [17]:
from langchain_chroma import Chroma

vectorstore = None # Initialize
if splits and embedding_function_nomic: # Check if prerequisites met
    try:
        collection_name = "research_docs_nomic"
        persist_directory = "./chroma_db_research_nomic" # Local path
        print(f"Attempting to create/load Chroma DB at {persist_directory}...")
        vectorstore = Chroma.from_documents(
            collection_name=collection_name,
            documents=splits,
            embedding=embedding_function_nomic, # Use the Nomic embeddings here
            persist_directory=persist_directory
        )
        print("Chroma vector store ready using Nomic Embeddings.")
    except Exception as e:
        print(f"ERROR creating/loading Chroma DB: {e}")
        vectorstore = None
else:
    print("Skipping Chroma DB creation (missing splits or Nomic embedding function).")

Attempting to create/load Chroma DB at ./chroma_db_research_nomic...
Chroma vector store ready using Nomic Embeddings.


In [18]:
if vectorstore:
    query = "When was FutureTech Corp founded?"
    try:
        print(f"\nPerforming similarity search for: '{query}'")
        search_results = vectorstore.similarity_search(query, k=2) # Get top 2
        print("\nTop 2 search results:")
        for i, result in enumerate(search_results, 1):
            print(f"--- Result {i} ---")
            print(f"Source: {result.metadata.get('source', 'Unknown')}")
            print(f"Content: {result.page_content[:200]}...") # Show snippet
            print("-" * 15)
    except Exception as e:
        print(f"Error during similarity search: {e}")
else:
    print("Skipping similarity search: No vector store.")


Performing similarity search for: 'When was FutureTech Corp founded?'

Top 2 search results:
--- Result 1 ---
Source: data/company_profile.pdf
Content: Company Profile
Company Name: FutureTech Corp
Founded: 2012
Headquarters: San Francisco, CA
Employees: 1,200+
Mission:
To innovate intelligent solutions that simplify lives and empower industries thro...
---------------
--- Result 2 ---
Source: data/employee_handbook.pdf
Content: Employee Handbook
Welcome to FutureTech Corp!
Working Hours:
- Standard hours are 9:00 AM to 6:00 PM, Monday to Friday.
- Employees may request flexible hours subject to manager approval.
Leave Policy...
---------------


In [19]:
retriever = None
if vectorstore:
    try:
        retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
        print("\nRetriever created.")
        # retrieved_docs = retriever.invoke(query) # Example invocation
        # print(f"Retriever found {len(retrieved_docs)} docs.")
    except Exception as e:
        print(f"Error creating retriever: {e}")
        retriever = None
else:
    print("Skipping retriever creation: No vector store.")


Retriever created.


In [20]:
from langchain.schema.runnable import RunnablePassthrough

if retriever and llm: # Check if prerequisites exist
    print("\nBuilding basic RAG chain...")
    template_rag = """Answer the question based only on the following context:
    {context}   
    Question: {question}
    Answer: """

    prompt_rag = ChatPromptTemplate.from_template(template_rag)

    def format_docs(docs): # Helper function
        return "\n\n".join(doc.page_content for doc in docs)
    
    rag_chain_basic = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt_rag
        | llm
        | output_parser # Use the StrOutputParser from earlier
    )
    
    print("Basic RAG chain created.")
    
    try:
        question_rag = "When was FutureTech Corp founded?"
        print(f"\nInvoking basic RAG chain for: '{question_rag}'")
        response_basic_rag = rag_chain_basic.invoke(question_rag)
        print(f"\nQuestion: {question_rag}")
        print(f"Answer: {response_basic_rag}")
    except Exception as e:
        print(f"Error invoking basic RAG chain: {e}")
else:
    print("Skipping basic RAG chain creation (missing retriever or llm).")


Building basic RAG chain...
Basic RAG chain created.

Invoking basic RAG chain for: 'When was FutureTech Corp founded?'

Question: When was FutureTech Corp founded?
Answer: 2012


In [21]:
from langchain_core.messages import AIMessage

# Simple list to simulate history for the notebook run
notebook_chat_history = []
question1_hist = "When was FutureTech Corp founded?"
# Assume 'response_basic_rag' holds the answer from the previous step
# In a real app, you'd use the RAG chain with history handling built-in
answer1_hist = response_basic_rag if 'response_basic_rag' in locals() else "FutureTech Corp was founded in 2012."

notebook_chat_history.extend([
    HumanMessage(content=question1_hist),
    AIMessage(content=answer1_hist)
])
print("Simulated chat history (list format):")
print(notebook_chat_history)

Simulated chat history (list format):
[HumanMessage(content='When was FutureTech Corp founded?', additional_kwargs={}, response_metadata={}), AIMessage(content='2012', additional_kwargs={}, response_metadata={})]


In [22]:
import sqlite3
from datetime import datetime
import uuid

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
    (id INTEGER PRIMARY KEY AUTOINCREMENT,
    session_id TEXT,
    user_query TEXT,
    gpt_response TEXT,
    model TEXT,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()

In [23]:
# Using the database functions defined earlier
print("\n--- Storing/Retrieving History with DB ---")
try:
    session_id_db = str(uuid.uuid4())
    print(f"Using session_id: {session_id_db}")

    # Simulate turn 1 from notebook cell 50
    q1_db = "What is FutureTech Corp?"
    # We need to run the full RAG chain again for a proper answer
    # For demonstration, let's use a placeholder answer
    a1_db = "FutureTech Corp, founded in 2012 and based in SF, creates AI solutions like Vision360."
    insert_application_logs(session_id_db, q1_db, a1_db, "llama3-8b-8192")
    print(f"Logged Turn 1 to DB.")

    # Simulate turn 2 from notebook cell 50
    q2_db = "What are their flagship product?"
    a2_db = "Their main products are Vision360 AI Camera, AutoInsights Dashboard, and RoboHR Assistant."
    insert_application_logs(session_id_db, q2_db, a2_db, "llama3-8b-8192")
    print(f"Logged Turn 2 to DB.")

    # Retrieve history from DB (like notebook cell 52)
    retrieved_db_history = get_chat_history(session_id_db)
    print("\nHistory retrieved from DB:")
    print(retrieved_db_history)

except Exception as e:
    print(f"Error interacting with chat history DB: {e}")


--- Storing/Retrieving History with DB ---
Using session_id: 52366755-336a-4ce9-a7b1-6300815eeb85
Logged Turn 1 to DB.
Logged Turn 2 to DB.

History retrieved from DB:
[{'role': 'human', 'content': 'What is FutureTech Corp?'}, {'role': 'ai', 'content': 'FutureTech Corp, founded in 2012 and based in SF, creates AI solutions like Vision360.'}, {'role': 'human', 'content': 'What are their flagship product?'}, {'role': 'ai', 'content': 'Their main products are Vision360 AI Camera, AutoInsights Dashboard, and RoboHR Assistant.'}]


In [24]:
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

if retriever and llm: # Check prerequisites
    print("\n--- Building History-Aware RAG Chain ---")
    # 1. Contextualizer Prompt: Reformulates question based on history
    contextualize_q_system_prompt = """Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."""
    contextualize_q_prompt = ChatPromptTemplate.from_messages([
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"), # Where history is injected
        ("human", "{input}"),
    ])

    # 2. History-Aware Retriever: Runs contextualizer then retrieves
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )
    print("History-aware retriever created.")

    # 3. QA Prompt: Includes context AND history for final answer
    qa_prompt_hist = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
        ("system", "Context: {context}"), # Retrieved docs go here
        MessagesPlaceholder("chat_history"), # History goes here
        ("human", "{input}") # Standalone question goes here
    ])

    # 4. QA Chain: Takes context/history/question -> generates answer
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt_hist)
    print("Question-answer chain created.")

    # 5. Final RAG Chain: Connects history_aware_retriever and question_answer_chain
    rag_chain_with_history = create_retrieval_chain(
        history_aware_retriever, question_answer_chain
    )
    print("Full history-aware RAG chain created.")

    try:
         # Example: See how the question gets reformulated
         reformulated_q = (contextualize_q_prompt | llm | StrOutputParser()).invoke(
             {"input": "Where is it headquartered?", "chat_history": notebook_chat_history}
         )
         print(f"\nReformulated question based on history: {reformulated_q}")

         # Example: See what docs the history-aware retriever fetches
         # relevant_docs_hist = history_aware_retriever.invoke(
         #     {"input": "Where is it headquartered?", "chat_history": notebook_chat_history}
         # )
         # print(f"\nDocs retrieved for reformulated question: {len(relevant_docs_hist)} docs")
    except Exception as e:
         print(f"Error demonstrating intermediate steps: {e}")

    try:
        print("\nInvoking full RAG chain with history...")
        question_follow_up = "Where is it headquartered?"
        answer_follow_up = rag_chain_with_history.invoke(
            {"input": question_follow_up, "chat_history": notebook_chat_history}
        )['answer']

        print(f"\nHuman: {question_follow_up}")
        print(f"AI: {answer_follow_up}")
        # Add to notebook history list
        # notebook_chat_history.extend([HumanMessage(content=question_follow_up), AIMessage(content=answer_follow_up)])
    except Exception as e:
        print(f"Error invoking history-aware RAG chain: {e}")

else:
    print("Skipping history-aware RAG chain (missing retriever or llm).")


--- Building History-Aware RAG Chain ---
History-aware retriever created.
Question-answer chain created.
Full history-aware RAG chain created.

Reformulated question based on history: What is the headquarters location of FutureTech Corp?

Invoking full RAG chain with history...

Human: Where is it headquartered?
AI: San Francisco, CA
